Selenium 구글 이미지 스크래핑
- 참고 : https://www.youtube.com/watch?v=pQ7dOg9c4NI

In [ ]:
# selenium 및 웹드라이버 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (139 kB/s)
Reading package lists... Done
Reading package lists... Done
Building

# 라이브러리 import

In [ ]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

# 크롤링 실행

## scroll_down() 함수 만들기
- 맨 밑에까지 스크롤을 한 다음 이미지를 저장해야 하니까 스크롤 하는 용도

In [ ]:
def scroll_down():
  scroll_count = 0
  print("[scroll_down() : 스크롤 다운 시작]")
  
  last_height = wd.execute_script("return document.body.scrollHeight")
  after_click = False

  while True:
    print(f"[스크롤 다운: {scroll_count}]")
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    scroll_count += 1
    time.sleep(1)
    
    new_height = wd.execute_script("return document.body.scrollHeight")
    
    if last_height == new_height:
      if after_click is True:  # 결과 더보기 클릭한 경우
        break
      else:
        try:
          more_button = wd.find_element_by_xpath("//*[@id=\"islmp\"]/div/div/div/div/div[5]/input")
          if more_button.is_displayed():
            more_button.click()
            after_click = True
        except NoSuchElementException as e:
          print(e)
          break
    last_height = new_height

## click_and_save 함수 만들기
- 썸네일 이미지 선택 후 원본 이미지 저장

In [ ]:
def click_and_save(dir_name, index, img, img_list_length):
    global scraped_count

    try:
      img.click()
      wd.implicitly_wait(3)
      src = wd.find_element_by_xpath("//*[@id=\"Sva75c\"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img").get_attribute("src")
      if src.split(".")[-1] == "png":
        urlretrieve(src, dir_name + "/" + str(scraped_count + 1) + ".png")
        print(f"{index+1} / {img_list_length} PNG 이미지 저장")
      else:
        urlretrieve(src, dir_name + "/" + str(scraped_count + 1) + ".jpg")
        print(f"{index+1} / {img_list_length} JPG 이미지 저장")
      
      scraped_count += 1
    except HTTPError as e:
      print(e)
      pass
    except ElementClickInterceptedException as e:
      print(e)
      pass
    

## scraping 함수 만들기

In [ ]:
def scraping(dir_name, query):
  global scraped_count

  url = f"https://www.google.com/search?q={query}"
  aa = wd.get(url)
  print("aa~~~~", aa)
  wd.maximize_window()

  scroll_down()
  div = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div')
  print("div는~~", divmod)
  img_list = div.find_elements_by_css_selector('//img[contains(@class,"rg_i Q4LuWd")]')
  print("img_list는~~", img_list)
  for index, img in enumerate(img_list):
    try:
      click_and_save(dir_name, index, img, len(img_list))
    except ElementClickInterceptedException as e:
      print(e)
      wd.execute_script("window.scrollTo(0, window.scrollY + 100")
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except NoSuchElementException as e:
      print(e)
      wd.execute_script("window.scrollTo(0, window.scrollY + 100")
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except ConnectionResetError as e:
      print(e)
      pass
    except URLError as e:
      print(e)
      pass
    except socket.timeout as e:
      print(e)
      pass
    except socket.gaierror as e:
      print(e)
      pass
    except ElementNotInteractableException as e:
      print(e)
      break
  try:
    print("[스크래핑 종료 (성공률: %.2f%%)]" % (scraped_count / len(img_list) * 100.0))
  except ZeroDivisionError as e:
    print(e)
  
  wd.quit()

## filter_and_remove 함수 만들기
- URL로 다운받으면 화질이 안 좋아지게 설정한 웹사이트의 이미지들 제거
- 일정 해상도 이하이거나 손상된 이미지 제거

In [ ]:
def filter_and_remove(dir_name, query, filter_size):
  filterd_count = 0

  for index, file_name in enumerate(os.listdir(dir_name)):
    try:
      file_path = os.path.join(dir_name, file_name)
      img = Image.open(file_path)

      if img.width < filter_size and img.height < filter_size:
        img.close()
        os.remove(file_path)
        print(f"{index} 이미지 제거")
        filtered_count += 1
    except OSError as e:
      print(e)
      os.remove(file_path)
      filtered_count += 1
  print(f"[이미지 제거 개수: {filtered_count}/{scraped_count}]")

In [ ]:
!rmdir 이광수

socket.setdefaulttimeout(30)

wd = webdriver.Chrome("chromedriver", options=chrome_options)
scraped_count = 0
path = "./"
query = input("검색어 입력: ")

dir_name = path + query
os.makedirs(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)

filter_and_remove(dir_name, query, 400)

검색어 입력: 이광수
[./이광수 디렉토리 생성]
aa~~~~ None
[scroll_down() : 스크롤 다운 시작]
[스크롤 다운: 0]
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="islmp"]/div/div/div/div/div[5]/input"}
  (Session info: headless chrome=86.0.4240.75)



NoSuchElementException: ignored

In [ ]:
# aa~~~가 None 이라고 나오는 걸로 보아
# url 이 잘못 되었음
# 전달되는 것이 없으니까
# 그 뒤에가 다 안된거임
# 스크롤다운도 안됬음
# 스크롤다운 0,, 1,, ... 쭉쭊 떠야하는데 안 뜸
# 그리고 그 xpath 다시 원래대로 하기